# Generate animations

Show the time series as animations of the spatial fluxes, and summarised as animated line plots

In [ ]:
import os
import sys
import xarray as xr
import numpy as np
import geopandas as gpd
from IPython.display import Image
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from odc.geo.xr import assign_crs
import matplotlib as mpl
from matplotlib.cm import ScalarMappable
import matplotlib.animation as animation

sys.path.append('/g/data/os22/chad_tmp/dea-notebooks/Tools')
from dea_tools.plotting import xr_animation

In [ ]:
from datacube.utils.dask import start_local_dask
client = start_local_dask(mem_safety_margin='2Gb')
client

In [ ]:
base = f'/g/data/xc0/project/AusEFlux/'
year_end = '2024'

## Open datasets

and reproject to limit data volume

In [ ]:
# ----NEE-------------------------------
folder = base+f'results/AusEFlux/NEE/'
files = [f'{folder}/{i}' for i in os.listdir(folder) if i.endswith(".nc")]
files.sort()
nee = xr.open_mfdataset(files).sel(time=slice('2003',year_end))
nee = assign_crs(nee, crs='EPSG:4326')
nee = nee['NEE_median']
nee.attrs['nodata'] = np.nan

# ----GPP-------------------------------
folder = base+f'results/AusEFlux/GPP/'
files = [f'{folder}/{i}' for i in os.listdir(folder) if i.endswith(".nc")]
files.sort()
gpp = xr.open_mfdataset(files).sel(time=slice('2003',year_end))
gpp = assign_crs(gpp, crs='EPSG:4326')
gpp = gpp['GPP_median']
gpp.attrs['nodata'] = np.nan

# ----ER-------------------------------
folder = base+f'results/AusEFlux/ER/'
files = [f'{folder}/{i}' for i in os.listdir(folder) if i.endswith(".nc")]
files.sort()
er = xr.open_mfdataset(files).sel(time=slice('2003',year_end))
er = assign_crs(er, crs='EPSG:4326')
er = er['ER_median']
er.attrs['nodata'] = np.nan


#### Zoom out to 2 km resolution to speed things up

In [ ]:
grid = nee.odc.geobox.zoom_out(factor=4)

nee = nee.odc.reproject(how=grid, resampling='average').compute()
gpp = gpp.odc.reproject(how=grid, resampling='average').compute()
er = er.odc.reproject(how=grid, resampling='average').compute()

## Image animation

### NEE

In [ ]:
path = f'{base}results/gifs/NEE_animation.gif'

imshow_kwargs = {'cmap': 'Spectral_r','vmin': -45, 'vmax': 45}

xr_animation(nee.to_dataset(name='NEE_median').rolling(time=3, min_periods=1).mean().compute(),
            bands=['NEE_median'],
            show_date='%b %Y',
            width_pixels=600,
            output_path=path,
            show_colorbar=True,
            colorbar_kwargs={'colors': 'black'},
            interval=100, 
            show_text='NEE gC m\N{SUPERSCRIPT TWO} mon⁻¹',
            imshow_kwargs=imshow_kwargs
            )

# Plot animation
plt.close()
Image(path, embed=True)

### GPP

In [ ]:
path = f'{base}results/gifs/GPP_animation.gif'

imshow_kwargs = {'cmap': 'gist_earth_r','vmin': 0, 'vmax': 150}

xr_animation(gpp.to_dataset(name='GPP_median').rolling(time=3, min_periods=1).mean().compute(),
            bands=['GPP_median'],
            show_date='%b %Y',
            width_pixels=600,
            output_path=path,
            show_colorbar=True,
            colorbar_kwargs={'colors': 'black'},
            interval=100, 
            show_text='GPP gC m\N{SUPERSCRIPT TWO} mon⁻¹',
            imshow_kwargs=imshow_kwargs
            )

# Plot animation
plt.close()
Image(path, embed=True)

## Animated line graph to accompany images

### Convert to PgC/year

In [ ]:
grid = nee.odc.geobox.to_crs('EPSG:3577')

nee = nee.odc.reproject(how=grid, resampling='bilinear').compute()
gpp = gpp.odc.reproject(how=grid, resampling='bilinear').compute()
er = er.odc.reproject(how=grid, resampling='bilinear').compute()

In [ ]:
area_per_pixel = nee.odc.geobox.resolution.x**2

nee = nee * area_per_pixel * 1e-15 * 12 # (pgC/year)
gpp = gpp * area_per_pixel * 1e-15 * 12 # (pgC/year)
er = er * area_per_pixel * 1e-15 * 12 # (pgC/year)

fire = xr.open_dataarray(f'{base}data/FireEmissions_10km_monthly.nc').sel(time=slice('2003',year_end))
fire = fire.sum(['x', 'y'])
fire = fire*12

In [ ]:
y_gpp=gpp.sum(['x', 'y']).rename('GPP PgC/month')#.rolling(time=3, min_periods=1).mean()
y_nee=nee.sum(['x', 'y']).rename('NEE PgC/month')#.rolling(time=3, min_periods=1).mean()
y_er=er.sum(['x', 'y']).rename('ER PgC/month')

y_fire = y_nee+fire

x=nee.time.values

### Rainfall anomalies

In [ ]:
folder = f'{base}data/interim_500m/rain'
files = [f'{folder}/{i}' for i in os.listdir(folder) if i.endswith(".nc")]
files.sort()
rain = xr.open_mfdataset(files).sel(time=slice('2003',year_end))
rain = assign_crs(rain, crs='EPSG:4326')
rain = rain['rain']
rain.attrs['nodata'] = np.nan
rain = rain.odc.reproject(how=grid, resampling='average').compute()
rain.name = 'rain'

In [ ]:
rain_clim_std = rain.sel(time=slice('2003', year_end)).groupby('time.month').std().compute()
rain_clim_mean = rain.sel(time=slice('2003', year_end)).groupby('time.month').mean().compute()
#get rid of zero values
# rain_clim_mean = xr.where(rain_clim_mean==0, 1, rain_clim_mean)

#standardized anom
def stand_anomalies(ds, clim_mean, clim_std):
    std_anom = xr.apply_ufunc(lambda x, m, s: (x - m) / s,
    ds.compute().groupby("time.month"),
    clim_mean, clim_std)
    return std_anom

rain_std_anom = stand_anomalies(rain, rain_clim_mean, rain_clim_std)
rain_std_anom = rain_std_anom.drop_vars('month')
rain_df = rain_std_anom.rename('rain').mean(['x','y']).rolling(time=3,min_periods=1).mean().to_dataframe().drop(['spatial_ref'], axis=1)

### Create animated lines

In [ ]:
def update_2lines(num, x, y, z, line1, line2):
    line1.set_data(x[:num], y[:num])
    line2.set_data(x[:num], z[:num])
    return [line1, line2]

def update(num, x, y, line):
    line.set_data(x[:num], y[:num])
    return line,

### GPP/ER

In [ ]:
export_line=path = f'{base}results/gifs/GPP_ER_line_animation.gif'

fig, ax = plt.subplots(figsize=(13,4))
# fig.set_size_inches(width * scale / 72, height * scale / 72, forward=True)
ax2 = ax.twinx()

line, = y_gpp.plot(ax=ax, label='GPP', linewidth=1.5, c='black')
line_er, = y_er.plot(ax=ax, label='ER', linewidth=1.5, c='red')

norm=plt.Normalize(-2,2)
cmap = mpl.colors.LinearSegmentedColormap.from_list("", ['saddlebrown','chocolate','white','darkturquoise','darkcyan'], N=256)

# Plot bars
bar = ax2.bar(rain_df.index, 1, color=cmap(norm(rain_df['rain'])), width=32)
sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax2,shrink=0.8, pad=0.01)
cbar.set_label('Rainfall Anomaly',labelpad=.5)
ax2.set_zorder(ax.get_zorder()-1)
ax2.set_ylabel('')
ax2.set_yticks([])
ax2.set_ylim([0, 1])
ax.margins(x=0)
ax2.margins(x=0)

ax.set_frame_on(False)
ax.set_yticklabels(['{:3.1f}'.format(x) for x in ax.get_yticks()])
ax.grid('off', which='major', axis='both', linestyle='--', linewidth=0.75)
ax.xaxis.set_major_locator(mdates.YearLocator(2))
ax.legend(loc='best', fontsize=14)
ax.set_ylabel("GPP/ER, PgC yr⁻¹", fontsize=14)
ax.set_xlabel('')
ax.set_title(None)
plt.tight_layout()
ani = animation.FuncAnimation(fig, update_2lines, len(x),
                          fargs=[x, y_gpp, y_er, line, line_er],
                          interval=100,
                          blit=True)

ani.save(export_line)
plt.close()
Image(export_line, embed=True)

### NEE

In [ ]:
export_line=path = f'{base}results/gifs/NEE_line_animation.gif'

fig, ax = plt.subplots(figsize=(13,4))
ax2 = ax.twinx()

line, = y_nee.plot(ax=ax, label='NEE', linewidth=1.5, c='black')
line_fire, = y_fire.plot(ax=ax, label='NEE+fire', linestyle='--', linewidth=1.5, c='purple')

norm=plt.Normalize(-2,2)
cmap = mpl.colors.LinearSegmentedColormap.from_list("", ['saddlebrown','chocolate','white','darkturquoise','darkcyan'], N=256)

# Plot bars
bar = ax2.bar(rain_df.index, 1, color=cmap(norm(rain_df['rain'])), width=32)
sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax2,shrink=0.8, pad=0.01)
cbar.set_label('Rainfall Anomaly',labelpad=.5)
ax2.set_zorder(ax.get_zorder()-1)
ax2.set_ylabel('')
ax2.set_yticks([])
ax2.set_ylim([0, 1])
ax.margins(x=0)
ax2.margins(x=0)

ax.set_frame_on(False)
ax.set_yticklabels(['{:3.1f}'.format(x) for x in ax.get_yticks()])
ax.set_title(None)
ax.grid('off', which='major', axis='both', linestyle='--', linewidth=0.75)
ax.xaxis.set_major_locator(mdates.YearLocator(2))
ax.legend(loc='best', fontsize=14)
ax.set_ylabel("NEE PgC yr⁻¹", fontsize=14)
ax.set_xlabel('')
ax.axhline(0, c='grey', linestyle='--')

plt.tight_layout()

ani = animation.FuncAnimation(fig, update_2lines, len(x),
                              fargs=[x, y_nee, y_fire, line, line_fire],
                              interval=100,
                              blit=True)

ani.save(export_line)
plt.close()
Image(export_line, embed=True)

In [ ]:
##